In [113]:
import json
from ultralytics import YOLO
import cv2
import numpy as np
import time
import csv
import torch
from matplotlib import pyplot as plt

"""conda install -c conda-forge opencv"""

'conda install -c conda-forge opencv'

In [114]:
def calculate_iou(box1, box2):
    """
    Calculate Intersection over Union (IoU) of two bounding boxes.

    Parameters:
    - box1: Tuple or List representing (x1, y1, x2, y2) of the first box.
    - box2: Tuple or List representing (x1, y1, x2, y2) of the second box.

    Returns:
    - IoU: Intersection over Union.
    """

    # Extract coordinates
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2

    # Calculate intersection coordinates
    x_intersection = max(0, min(x2_1, x2_2) - max(x1_1, x1_2))
    y_intersection = max(0, min(y2_1, y2_2) - max(y1_1, y1_2))

    # Calculate areas of boxes and intersection
    area_box1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area_box2 = (x2_2 - x1_2) * (y2_2 - y1_2)
    area_intersection = x_intersection * y_intersection

    # Calculate Union
    area_union = area_box1 + area_box2 - area_intersection

    # Calculate IoU
    iou = area_intersection / (area_union + 1e-10)  # Add a small epsilon to avoid division by zero

    return iou

# Example usage:
box1 = (0, 0, 4, 4)
box2 = (2, 2, 6, 6)

iou = calculate_iou(box1, box2)
print(f"IoU: {iou}")


IoU: 0.14285714285663267


In [115]:
class Back_number_recogniser:
    def __init__(self):
        self.numbers = []
        self.ttl = 3
        pass
    def read_participants(self,classesFilePath):
        # Opening JSON file
        f = open(classesFilePath)
        
        
        self.number_to_name = {}
        # returns JSON object as 
        # a dictionary
        self.participants = json.load(f)
        for i in self.participants:
            self.number_to_name[i["race_rider_bib_number"]] = i["rider"]["first_name"] + " "+ i["rider"]["last_name"]
        # Closing file
        f.close()
        
    def load_detection_model(self, model_path = "C:/Users/Eigenaar/Documents/school_Quinten/unief_2023_2024/Masterthesis/implementatie/number_detection/number_detection_10epochs.pt"):
        device = torch.device("cuda")
        self.detection_model = YOLO(model_path)
        self.detection_model.to(device)
        
    def load_recognition_model(self, model_path = "C:/Users/Eigenaar/Documents/school_Quinten/unief_2023_2024/Masterthesis/implementatie/number_recognition/number_recognition_10epochs.pt"):
        device = torch.device("cuda")
        self.recognition_model = YOLO(model_path)
        self.recognition_model.to(device)
        
    def detect_numbers(self,image):
        
        results = self.detection_model(image,verbose=False)
        return results
    
    def add_bb(self, image, boxes):
        ret_image = np.copy(image)
        for i in boxes:
            top = tuple([int(i[0]),int(i[1])])
            bottom = tuple([int(i[2]),int(i[3])])
            cv2.rectangle(ret_image,top,bottom, (255,0,0), 2)
        return ret_image
    
    def crop_image(self, image, box):
        image_copy = np.copy(image)
        left = int(box[0].item())
        right = int(box[2].item())
        top = int(box[1].item())
        bottom = int(box[3].item())
        cropped_image = image_copy[ top:bottom, left:right] # Slicing to crop the image
        return cropped_image
    
    def deinterlace_image(self, image):
        ret_image = np.copy(image)
        for i in range(len(ret_image)//2):
            ret_image[2*i+1] = ret_image[2*i]
        return ret_image
    
    def recognise_numbers(self, image, bboxes):
        cropped_images = []
        boxes = []
        for i in range(len(bboxes)):
            cropped_image = self.crop_image(image, bboxes[i])
            deinterlaced_image = self.deinterlace_image(cropped_image)
            cropped_images.append(deinterlaced_image)
        
        results = self.recognition_model(cropped_images, verbose=False)
        detected_numbers = []
        for i in range(len(results)):
            result = results[i].boxes


            predicted_digits = []
            average_x = []
            digits_to_discard = []
            for l in range(len(result.cls)):
                for j in range(l+1, len(result.cls)):
                    box1 = (result.xyxy[l][0].item(), result.xyxy[l][1].item(), result.xyxy[l][2].item(), result.xyxy[l][3].item())
                    box2 = (result.xyxy[j][0].item(), result.xyxy[j][1].item(), result.xyxy[j][2].item(), result.xyxy[j][3].item())
                    iou = calculate_iou(box1, box2)
                    if iou > 0.5:
                        if result.conf[l].item() > result.conf[j].item():
                            digits_to_discard.append(j)
                        else:
                            digits_to_discard.append(l)
                        
            for j in range(len(result.cls)):
                if j not in digits_to_discard:
                    predicted_digits.append(int(result.cls[j].item()))
                    average_x.append((result.xyxy[j][2].item()+result.xyxy[j][0].item())/2)
                
            predicted_digits = [x for _,x in sorted(zip(average_x,predicted_digits))]
            predicted_number ="".join([str(i) for i in predicted_digits])
            if predicted_number != "":
                detected_numbers.append(predicted_number)
                boxes.append(bboxes[i])
            
            """numbers = []
            coords = []
            for j in range(len(result.cls)):
                numbers.append(result.cls[j].item())
                coords.append(result.xyxy[j][0].item())
            numbers_sorted = [x for _,x in sorted(zip(coords,numbers))]
            number = ''
            for j in numbers_sorted:
                number+= str(int(j))
            detected_numbers.append(number)
            print(f"numbers in cropped_image: {numbers} and with coordinates {coords}")"""
        return boxes, detected_numbers
        
        
    def annotate_image(self, image, boxes,numbers,threshold = 0.5):
        
        imH, imW, imC = image.shape
        annotated_image = self.add_bb(image, boxes)
        
        if len(boxes) != 0:
            for i in range(len(boxes)):
                
                classLabelText = numbers[i]
                left = int(boxes[i][0])
                top = int(boxes[i][1])
                cv2.putText(annotated_image, classLabelText, (left, top-10), cv2.FONT_HERSHEY_PLAIN, 1, [0,0,255],2)
                
        return annotated_image

    def search_names(self, numbers):
        names= []
        for i in numbers:
            if int(i) in self.number_to_name:
                names.append(self.number_to_name[int(i)])
        return names

    def match_box(self, box, number):
        if len(self.numbers) != 0:
            for key in self.numbers:
                if calculate_iou(box, key[1]) >0.2:
                    key[0] = self.ttl
                    key[1] = box
                    if number in key[2]:
                        key[2][number] +=0.5
                    for value in key[2].keys():
                        if number in value:
                            key[2][value] += 0.5
                    if number not in key[2]:
                        key[2][number] = 1
                    return 
        if len(number) <=3:
            self.numbers.append([self.ttl, box,{number:1}])
        else:
            self.numbers.append([self.ttl, box,{number[0:3]:1}])
        return 

    def remove_boxes(self):
        print(len(self.numbers))
        self.numbers = [[item[0]-1, item[1], item[2]] for item in self.numbers if item[0]-1 >0]
        print(len(self.numbers))
            
        
                
        
    def predict_image(self,image):
        results = self.detect_numbers(image)
        boxes = results[0].boxes.xyxy
        if len(boxes) != 0:
            boxed_image = self.add_bb(image, boxes)
            boxes, detected_numbers = self.recognise_numbers(image,boxes)
            return boxes, detected_numbers
        else:
            return [], []
    
    def process_image(self,imagePath):
        image = cv2.imread(imagePath)
        boxes, numbers = self.predict_image(image)
        annotated_image = self.annotate_image(image, boxes, detected_numbers)
        # cv2.imshow("result", annotated_image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        return annotated_image
    
    def process_video(self,videoPath,outputfile,startpoint = 0):
        cap = cv2.VideoCapture(videoPath)
        cap.set(cv2.CAP_PROP_POS_FRAMES, startpoint)
        # Check if the video capture was successful
        if not cap.isOpened():
            print("Error: Could not open the video file.")
            exit()
        ret, frame = cap.read()
        frame_width = int(frame.shape[1]) 
        frame_height = int(frame.shape[0])
        size = (frame_width, frame_height)
        result = cv2.VideoWriter(outputfile,  cv2.VideoWriter_fourcc(*'MJPG'), 25, size) 
        # used to record the time when we processed last frame 
        prev_frame_time = time.time() 
        framenr = 0
        # used to record the time at which we processed current frame 
        new_frame_time = 0
        with open(outputfile.split('.')[0] + '.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            field = ["framenr", "riders", "numbers", "boxes"]
            writer.writerow(field)
     
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                boxes, numbers = self.predict_image(frame)
                # match a bounding box to the already known ones
                if len(boxes)!=0:
                    for i in range(len(boxes)):
                        left = int(boxes[i][0].item())
                        right = int(boxes[i][2].item())
                        top = int(boxes[i][1].item())
                        bottom = int(boxes[i][3].item())
                        box = (left, top, right, bottom)
                        self.match_box(box, numbers[i])
                final_boxes = []
                final_numbers = []
                for i in self.numbers:
                    final_boxes.append(i[1])
                    final_numbers.append(max(i[2], key=i[2].get))
                names = self.search_names(final_numbers)
                annotated_image = self.annotate_image(frame, final_boxes, final_numbers)
                
                # font which we will be using to display FPS 
                font = cv2.FONT_HERSHEY_SIMPLEX 
                # time when we finish processing for this frame 
                new_frame_time = time.time() 
          
                # Calculating the fps 
              
                # fps will be number of frame processed in given time frame 
                # since their will be most of time error of 0.001 second 
                # we will be subtracting it to get more accurate result 
                fps = 1/(new_frame_time-prev_frame_time) 
                prev_frame_time = new_frame_time 
              
                # converting the fps into integer 
                fps = int(fps) 
              
                # converting the fps to string so that we can display it on frame 
                # by using putText function 
                fps = str(fps) 
              
                # putting the FPS count on the frame 
                cv2.putText(annotated_image, fps, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA) 
                """plt.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
                plt.show()"""
                self.remove_boxes()
                framenr+=1
                print(framenr)
                writer.writerow([framenr, names, final_numbers, final_boxes])
                """if framenr >10:
                    break"""
                result.write(annotated_image)
                #cv2.imshow('frame', annotated_image)
                
                #if cv2.waitKey(1) == ord('q'):
                 #   break
        cap.release()
        result.release()
        #cv2.destroyAllWindows()

In [ ]:

#image = "C:/Users/Eigenaar/Documents/school_Quinten/unief_2023_2024/Masterthesis/data/wva_foto.png"
#image = "C:/Users/Eigenaar/Documents/school_Quinten/unief_2023_2024/Masterthesis/implementatie/number_detection/data/images/e1948ebd-759b-11ee-bced-fcb3bc5cd112.jpg"
#image = "C:/Users/Eigenaar/Documents/school_Quinten/unief_2023_2024/Masterthesis/implementatie/number_detection/data/images/d2293da5-759a-11ee-b39f-fcb3bc5cd112.jpg"
#image = "C:/Users/Eigenaar/Documents/school_Quinten/unief_2023_2024/Masterthesis/implementatie/number_detection/data/images/d7a17471-759a-11ee-b7c3-fcb3bc5cd112.jpg"
#video = "C:/Users/Eigenaar/Documents/school_Quinten/unief_2023_2024/Masterthesis/implementatie/mt23/data/cut_rvv_1.mp4"
video = "data/cut_rvv_1.mov"
participant_file = "data/RVV23participants.json"
#participant_file = "data/OHN23 participants.json"
detection_path = "number_detection/runs/detect/train6/weights/best.pt" 
recognition_path = "number_recognition/runs/detect/train4/weights/best.pt"
outputfile = "data/results__with_linking_cut_rvv_1.mp4"

recogniser = Back_number_recogniser()
recogniser.read_participants(participant_file)
recogniser.load_detection_model(detection_path)
recogniser.load_recognition_model(recognition_path)
recogniser.process_video(video,outputfile)


OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


6
6
1
6
6
2
6
6
3
6
6
4
6
6
5
6
6
6
6
6
7
6
6
8
6
6
9
6
6
10
6
6
11
6
6
12
6
6
13
6
6
14
6
6
15
6
6
16
6
6
17
6
6
18
6
6
19
6
6
20
6
6
21
6
6
22
6
6
23
6
6
24
6
6
25
6
6
26
6
6
27
6
6
28
6
6
29
6
6
30
6
5
31
5
5
32
6
6
33
6
6
34
6
6
35
6
6
36
6
6
37
6
6
38
6
6
39
6
6
40
6
6
41
6
6
42
6
6
43
6
6
44
6
2
45
3
3
46
3
3
47
5
5
48
6
6
49
6
6
50
6
6
51
6
6
52
6
6
53
6
6
54
6
6
55
6
6
56
6
6
57
6
6
58
6
6
59
6
6
60
6
6
61
6
5
62
5
3
63
5
5
64
5
5
65
5
5
66
5
5
67
6
6
68
6
6
69
6
6
70
6
5
71
5
5
72
5
5
73
5
5
74
5
5
75
5
5
76
5
4
77
4
1
78
3
2
79
5
5
80
5
5
81
6
5
82
5
5
83
5
5
84
5
5
85
5
5
86
5
5
87
6
6
88
6
6
89
6
6
90
6
6
91
6
6
92
6
4
93
4
2
94
2
1
95
2
2
96
4
4
97
5
5
98
6
6
99
6
6
100
6
6
101
6
5
102
6
2
103
4
3
104
4
4
105
4
3
106
3
2
107
3
2
108
3
3
109
4
4
110
4
4
111
5
3
112
4
4
113
4
4
114
4
4
115
4
4
116
5
5
117
6
6
118
6
6
119
6
4
120
10
6
121
6
6
122
6
6
123
6
6
124
6
6
125
6
6
126
6
6
127
6
6
128
6
6
129
6
6
130
6
3
131
7
6
132
6
6
133
6
6
134
6
6
135
6
5
136
5
5
137
5
5
138
6
6